In [5]:

import os
import nltk
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import hvplot.pandas
%matplotlib inline
# Load .env enviroment variables
load_dotenv()


True

In [6]:
# Reading the News API key enviroment variable
api_key = os.getenv("Api_keys.env")

In [7]:


# Set News API Key
#newsapi = NewsApiClient(api_key=os.environ["news_api"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

#api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [8]:
load_dotenv(dotenv_path="Api_keys.env")
api_key = os.getenv("news_api")
api_key

'22b1baf9e3d84f549a60cd9ca7881995'

In [9]:
newsapi = NewsApiClient(api_key=api_key)
newsapi

In [10]:
corp_news = newsapi.get_everything(q="AAL", language="en")
corp_news["totalResults"]
corp_df = pd.DataFrame.from_dict(corp_news["articles"])
corp_df

,author,content,description,publishedAt,source,title,url,urlToImage
0,Pete Muntean,Washington (CNN Business)After air travel brok...,After air travel broke a pandemic record on We...,2020-12-25T13:41:29Z,"{'id': 'cnn', 'name': 'CNN'}",Nearly a million people flew in America on Chr...,https://www.cnn.com/2020/12/25/business/tsa-ai...,https://cdn.cnn.com/cnnnext/dam/assets/2012231...
1,Claudia Assis,It was housekeeping day for JPMorgan analysts ...,"United, JetBlue and Spirit shares get a downgr...",2020-12-16T17:53:00Z,"{'id': None, 'name': 'MarketWatch'}",": United, JetBlue stocks get downgrades from J...",https://www.marketwatch.com/story/united-jetbl...,https://images.mktw.net/im-240768/social
2,Tim Dunn,As covid-19 continues to negatively impact the...,The covid-19 financial crisis facing airlines ...,2020-12-17T13:35:35Z,"{'id': None, 'name': 'Seeking Alpha'}",American Airlines' Market Losses Will Aid Comp...,https://seekingalpha.com/article/4395151-ameri...,https://static.seekingalpha.com/uploads/2020/1...
3,Mark Vickery,Non-profit humanitarian agency World Vision Un...,American Airlines (AAL) put its first commerci...,2020-12-29T22:38:10Z,"{'id': None, 'name': 'Yahoo Entertainment'}",Markets Can't Sustain Record Highs; 737 MAX Fl...,https://finance.yahoo.com/news/markets-cant-su...,https://s.yimg.com/uu/api/res/1.2/nYSrGTfdEN.B...
4,Zacks Equity Research,"In the latest trading session, American Airlin...",American Airlines (AAL) closed at $15.66 in th...,2020-12-24T22:45:10Z,"{'id': None, 'name': 'Yahoo Entertainment'}",American Airlines (AAL) Stock Sinks As Market ...,https://finance.yahoo.com/news/american-airlin...,https://s.yimg.com/uu/api/res/1.2/OujcczywIVmb...
5,Maharathi Basu,"In the past week, American Airlines AAL restar...",American Airlines (AAL) resumes plying the Boe...,2020-12-31T17:50:05Z,"{'id': None, 'name': 'Yahoo Entertainment'}",Airline Stock Roundup: AAL Resumes Boeing 737 ...,https://finance.yahoo.com/news/airline-stock-r...,https://s.yimg.com/uu/api/res/1.2/nYSrGTfdEN.B...
6,Abigail Fisher,Out of thousands of stocks that are currently ...,Out of thousands of stocks that are currently ...,2020-12-21T19:18:47Z,"{'id': None, 'name': 'Yahoo Entertainment'}",Is AAL A Good Stock To Buy According To Hedge ...,https://finance.yahoo.com/news/aal-good-stock-...,https://s.yimg.com/uu/api/res/1.2/Y2wuJJU0o5AT...
7,newsfeedback@fool.com (Lou Whiteman),What happened\r\n2020 was a miserable year for...,"A long, miserable year comes to an upbeat end.",2021-01-06T01:24:36Z,"{'id': None, 'name': 'Motley Fool'}",Why Shares of American Airlines Gained Altitud...,https://www.fool.com/investing/2021/01/05/why-...,https://g.foolcdn.com/editorial/images/607343/...
8,Mark Hulbert,The most heavily shorted stocks underperform t...,Short interest in Tesla is not as high as inve...,2020-12-16T11:55:04Z,"{'id': None, 'name': 'MarketWatch'}",FA Center: Why Tesla bulls are in the driver’s...,https://www.marketwatch.com/story/why-tesla-bu...,https://s.marketwatch.com/public/resources/ima...
9,"Ted Reed, Senior Contributor, \n Ted Reed, Sen...",American Airlines jets sit at the gate in Miam...,Airline industry veterans expect business trav...,2020-12-30T15:09:12Z,"{'id': None, 'name': 'Forbes'}","Airline Shares Will End 2020 Down, But The Sky...",https://www.forbes.com/sites/tedreed/2021/12/3...,https://thumbor.forbes.com/thumbor/fit-in/1200...


In [69]:
def get_news(company):
    corp_news = newsapi.get_everything(q=company, language="en")
    corp_news["totalResults"]
    corp_df = pd.DataFrame.from_dict(company_news["articles"])
    corp_sentiments = []

    for article in corp_news["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
            
            corp_sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu
                
            })
            
        except AttributeError:
            pass

In [ ]:
# Create DataFrame
company_df = pd.DataFrame(corp_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
company_df = company_df[cols]
company_df.sort_values(by=['date'], ascending=False, inplace=True)
company_df.reset_index(inplace=True, drop=True)
company_df = company_df.drop(["text", "positive", "negative", "neutral"], axis=1)
company_df = company_df.set_index('date')

return company_df

In [ ]:
companies = ["AAL"]

for company in companies:
    print(company)
    print(get_news(company))